In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import spacy
from spacy.lang.en import English
from spacy import displacy
import pandas as pd

In [ ]:
df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

In [ ]:
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-base')
def process_data(row):
    text = row['review']
    text = str(text)
    text = ' '.join(text.split())

    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=128)

    label = 0
    if row['sentiment'] == 'positive':
        label += 1

    encodings['label'] = label
    encodings['text'] = text

    return encodings

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
train_data = []
test_data = []


for i in range(len(df[:30000])):
    train_data.append(process_data(df.iloc[i]))
for i in range(len(df_test[:20000])):
    test_data.append(process_data(df_test.iloc[i]))

In [ ]:
train_df=pd.DataFrame(train_data)
test_df=pd.DataFrame(test_data)

In [ ]:
import pyarrow as pa
from datasets import Dataset

train_datatest = Dataset(pa.Table.from_pandas(train_df))
test_datatest = Dataset(pa.Table.from_pandas(test_df))

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=5)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(output_dir="test_trainer")

In [ ]:
!pip install evaluate

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=df,
    eval_dataset=df_test,
    compute_metrics=compute_metrics,
)

In [ ]:
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import numpy as np
!pip install evaluate
import evaluate  # Assuming this is a custom evaluation script
df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Function to process each row of the DataFrame
def process_data(row):
    text = str(row['review']).strip()  # Remove leading/trailing spaces
    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    label = 1 if row['sentiment'] == 'positive' else 0  # Encode label
    encodings['labels'] = label
    return encodings

# Process train and test data
train_data = []
test_data = []

# Assuming df and df_test are already defined DataFrames
for i in range(len(df)):
    train_data.append(process_data(df.iloc[i]))

for i in range(len(df_test)):
    test_data.append(process_data(df_test.iloc[i]))

# Convert processed data to DataFrames
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Convert DataFrames to Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Load model
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)  # Assuming binary classification

# Define evaluation metric
metric = evaluate.load("accuracy")  # Assuming this is a custom evaluation function

# Function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Training arguments
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,  # Example number of epochs
    per_device_train_batch_size=8,  # Example batch size
    per_device_eval_batch_size=8,  # Example batch size
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=500,  # Log every 500 steps
    save_total_limit=2,  # Example number of checkpoints to save
    load_best_model_at_end=True,  # Load the best model at the end of training
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.460800,0.382522,0.864100
2,0.342400,0.350350,0.887850
3,0.252100,0.399250,0.896900


TrainOutput(global_step=11250, training_loss=0.35174189453125, metrics={'train_runtime': 2785.4578, 'train_samples_per_second': 32.311, 'train_steps_per_second': 4.039, 'total_flos': 5919998745600000.0, 'train_loss': 0.35174189453125, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.35034990310668945,
 'eval_accuracy': 0.88785,
 'eval_runtime': 150.3638,
 'eval_samples_per_second': 133.011,
 'eval_steps_per_second': 16.626,
 'epoch': 3.0}

In [ ]:
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset
import numpy as np
!pip install evaluate
import evaluate  # Assuming this is a custom evaluation script
df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

# Function to process each row of the DataFrame
def process_data(row):
    text = str(row['review']).strip()  # Remove leading/trailing spaces
    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    label = 1 if row['sentiment'] == 'positive' else 0  # Encode label
    encodings['labels'] = label
    return encodings

# Process train and test data
train_data = []
test_data = []

# Assuming df and df_test are already defined DataFrames
for i in range(len(df)):
    train_data.append(process_data(df.iloc[i]))

for i in range(len(df_test)):
    test_data.append(process_data(df_test.iloc[i]))

# Convert processed data to DataFrames
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Convert DataFrames to Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Load model
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=2)  # Assuming binary classification

# Define evaluation metric
metric = evaluate.load("accuracy")  # Assuming this is a custom evaluation function

# Function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Training arguments
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,  # Example number of epochs
    per_device_train_batch_size=16,  # Example batch size
    per_device_eval_batch_size=16,  # Example batch size
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=500,  # Log every 500 steps
    save_total_limit=2,  # Example number of checkpoints to save
    load_best_model_at_end=True,  # Load the best model at the end of training
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.368300,0.300600,0.886300
2,0.272000,0.357673,0.888300
3,0.204900,0.445707,0.896450
4,0.139400,0.444421,0.894100
5,0.090300,0.470566,0.896950


TrainOutput(global_step=9375, training_loss=0.21497255208333332, metrics={'train_runtime': 4157.2694, 'train_samples_per_second': 36.081, 'train_steps_per_second': 2.255, 'total_flos': 9866664576000000.0, 'train_loss': 0.21497255208333332, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.3006000220775604,
 'eval_accuracy': 0.8863,
 'eval_runtime': 146.7521,
 'eval_samples_per_second': 136.284,
 'eval_steps_per_second': 8.518,
 'epoch': 5.0}

In [2]:
!pip install evaluate
!pip install datasets
!pip install accelerate -U

In [1]:
import pandas as pd
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import numpy as np
import pyarrow as pa

from datasets import Dataset
import evaluate  # Assuming this is a custom evaluation script
df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

# Function to process each row of the DataFrame
def process_data(row):
    text = str(row['review']).strip()  # Remove leading/trailing spaces
    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    label = 1 if row['sentiment'] == 'positive' else 0  # Encode label
    encodings['labels'] = label
    return encodings

# Process train and test data
train_data = []
test_data = []

# Assuming df and df_test are already defined DataFrames
for i in range(len(df)):
    train_data.append(process_data(df.iloc[i]))

for i in range(len(df_test)):
    test_data.append(process_data(df_test.iloc[i]))

# Convert processed data to DataFrames
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

# Convert DataFrames to Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Load model
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)  # Assuming binary classification

# Define evaluation metric
metric = evaluate.load("accuracy")  # Assuming this is a custom evaluation function

# Function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Training arguments
training_args = TrainingArguments(
    output_dir="test_trainer",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=5,  # Example number of epochs
    per_device_train_batch_size=8,  # Example batch size
    per_device_eval_batch_size=8,  # Example batch size
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=500,  # Log every 500 steps
    save_total_limit=2,  # Example number of checkpoints to save
    load_best_model_at_end=True,  # Load the best model at the end of training
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

# Start training
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.395700,0.436416,0.859550
2,0.270000,0.402102,0.874150
3,0.143900,0.670186,0.873950
4,0.058200,0.754608,0.877100
5,0.020200,0.912367,0.876500


TrainOutput(global_step=18750, training_loss=0.17759222981770834, metrics={'train_runtime': 2404.575, 'train_samples_per_second': 62.381, 'train_steps_per_second': 7.798, 'total_flos': 4967527449600000.0, 'train_loss': 0.17759222981770834, 'epoch': 5.0})

In [2]:
trainer.evaluate()

{'eval_loss': 0.4021022915840149,
 'eval_accuracy': 0.87415,
 'eval_runtime': 80.6436,
 'eval_samples_per_second': 248.005,
 'eval_steps_per_second': 31.001,
 'epoch': 5.0}

In [ ]:
# Perform sentiment analysis
y_pred_R=[]
y_pred_D=[]
y_pred_G=[]
for i in range(0,5):

    # RoBERTa
    sentiment_roberta = analyze_sentiment(df.iloc[i,0], roberta_model, roberta_tokenizer)
    y_pred_R.append(sentiment_roberta)
    print(sentiment_roberta,df.iloc[i,1])
    # DistilBERT
    sentiment_distilbert = analyze_sentiment(df.iloc[i,0], distilbert_model, distilbert_tokenizer)
    y_pred_D.append(sentiment_distilbert)
    print(sentiment_distilbert,df.iloc[i,1])
    # GPT-2
    sentiment_gpt2 = analyze_sentiment(df.iloc[i,0], gpt2_model, gpt2_tokenizer)
    y_pred_G.append(sentiment_gpt2)
    print(sentiment_gpt2,df.iloc[i,1])
print("Accuracy_Roberta:",sum(y_pred_R==df.iloc[0:5,1])/5)
print("Accuracy_Distilbert:",sum(y_pred_D==df.iloc[0:5,1])/5)
print("Accuracy_GPT-2:",sum(y_pred_G==df.iloc[0:5,1])/5)



positive negative
negative negative
negative negative
positive positive
negative positive
negative positive
positive positive
negative positive
negative positive
positive negative
negative negative
negative negative
positive negative
negative negative
negative negative
Accuracy_Roberta: 0.4
Accuracy_Distilbert: 0.6
Accuracy_GPT-2: 0.6
